# RAG Using Azure AI Agent Service & Semantic Kernel

This code snippet demonstrates how to create and manage an Azure AI agent for retrieval-augmented generation (RAG) using the `Azure AI Agent Service` and `Semantic Kernel`. The agent processes user queries based on the retrieved context and provides accurate responses accordingly.

## Initializing the Environment

SQLite Version Fix
If you encounter the error:
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```

Uncomment this code block at the start of your notebook:

In [46]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importing Packages
The following code imports the necessary packages:

In [47]:
# Azure imports for project client and credentials
from azure.identity.aio import AzureCliCredential

# Agent Framework for Azure AI Agent Service file management
from agent_framework.azure import AzureAIAgentClient
from agent_framework import HostedVectorStoreContent

# Semantic Kernel imports
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

# Import os to access environment variables
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')

True

# Retrieval-Augmented Generation with Semantic Kernel & Azure AI Agent Service

This sample demonstrates how to use the **Azure AI Agent Service** to perform **Retrieval-Augmented Generation (RAG)** by combining a language model with domain-specific context from an uploaded document.

### How It Works

1. **Document Upload**: A markdown file (document.md) containing information (Contoso's travel insurance policy) is uploaded to the agent service.

2. **Vector Store Creation**: The document is indexed into a vector store to enable semantic search over its contents.

3. **Agent Configuration**: An agent is instantiated using the `gpt-4o` model with the following strict instructions:
   - Only answer questions based on retrieved content from the document.
   - Decline to answer if the question is out of scope.

4. **File Search Tool Integration**: The `FileSearchTool` is registered with the agent, enabling the model to search and retrieve relevant snippets from the indexed document during inference.

5. **User Interaction**: Users can ask questions. If relevant information is found in the document, the agent generates a grounded answer.  
   If not, the agent explicitly responds that the document does not contain sufficient information.


### Main Function


Make sure to first run `az login` using the Azure CLI so that the proper authentication context is provided while using the `DefaultAzureCredential`. The Azure AI Agent Service does not use API keys.

In [48]:
class AzureAIFileSearchPlugin:
    """Plugin to search documents using Azure AI Agent Service file search."""

    def __init__(self, azure_client: AzureAIAgentClient, vector_store: HostedVectorStoreContent):
        self.azure_client = azure_client
        self.vector_store = vector_store
        self.agent = None

    async def initialize(self):
        """Initialize the Azure AI Agent with file search capability."""
        from agent_framework import HostedFileSearchTool

        file_search = HostedFileSearchTool(inputs=self.vector_store)

        # Create agent with stricter instructions
        self.agent = self.azure_client.create_agent(
            name="FileSearchAgent",
            instructions="""You MUST search the documents for information.
            If the search returns no results, respond:
            "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
            Do NOT use your general knowledge.""",
            tools=[file_search],
            tool_choice="required"  # Force the tool to be used
        )

        print(f"Initialized Azure AI Agent for file search")

    @kernel_function(
        name="search_documents",
        description="REQUIRED: Search uploaded documents for relevant information. This function MUST be called before answering ANY question."
    )
    async def search_documents(self, query: str) -> str:
        """Search documents using Azure AI file search."""
        if not self.agent:
            return "Agent not initialized."


        # Use the agent to search documents
        response_text = ""
        async for chunk in self.agent.run_stream(
            query,
            tool_resources={"file_search": {"vector_store_ids": [self.vector_store.vector_store_id]}}
        ):
            if chunk.text:
                response_text += chunk.text

        if not response_text:
            return "NO_INFORMATION_FOUND"

        return response_text


In [49]:
async def create_vector_store(client: AzureAIAgentClient) -> tuple[str, HostedVectorStoreContent]:
    """Create a vector store with sample documents."""
    file_path = './document.md'
    agents_client = client.agents_client
    file = await agents_client.files.upload_and_poll(file_path=file_path, purpose="assistants")
    print(f"Uploaded file, file ID: {file.id}")

    vector_store = await agents_client.vector_stores.create_and_poll(
        file_ids=[file.id],
        name="semantic_kernel_vectorstore"
    )
    print(f"Created vector store, ID: {vector_store.id}\n")

    return file.id, HostedVectorStoreContent(vector_store_id=vector_store.id)


In [50]:
async def main():
    async with (
        AzureCliCredential() as creds,
        AzureAIAgentClient(
            async_credential=creds,
            project_endpoint=os.environ.get("AZURE_AI_AGENT_PROJECT_ENDPOINT"),
            model_deployment_name=os.environ.get("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"),
        ) as azure_client,
    ):
        # Upload file and create vector store using AzureAIAgentClient
        file_id, vector_store = await create_vector_store(azure_client)

        # Create Azure AI file search plugin
        file_search_plugin = AzureAIFileSearchPlugin(azure_client, vector_store)
        await file_search_plugin.initialize()

        # Create Semantic Kernel chat completion service
        chat_service = AzureChatCompletion(
            deployment_name=os.environ.get("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"),
            endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
            api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
        )

        # Stricter instructions with explicit function calling requirement
        agent = ChatCompletionAgent(
            service=chat_service,
            name="RAGAgent",
            instructions="""
                CRITICAL RULES:
                1. You MUST call the search_documents function for EVERY question.
                2. If search_documents returns "NO_INFORMATION_FOUND", respond EXACTLY:
                   "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
                3. You are FORBIDDEN from using your general knowledge or pre-trained information.
                4. You can ONLY answer using information explicitly returned by search_documents.
                5. If you violate these rules, your response will be considered invalid.
                """,
        )

        # Enable automatic function calling with required behavior
        from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
        from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import AzureChatPromptExecutionSettings

        settings = AzureChatPromptExecutionSettings(
            function_choice_behavior=FunctionChoiceBehavior.Auto()  # Force function calls when needed
        )

        # Add the plugin to the agent's kernel
        agent.kernel.add_plugin(file_search_plugin, "FileSearch")

        # Create thread for conversation
        thread = None

        print("Semantic Kernel agent created with STRICT file search enforcement.\n")

        # Example queries
        queries = [
            "Can you explain Contoso's travel insurance coverage?",
            "What is a Neural Network?"
        ]
        
        try:
            for query in queries:
                print("=" * 50)
                print(f"User question: {query}\n")
                print("Agent response: ")

                # Stream agent response with settings to enforce function calling
                async for response in agent.invoke_stream(
                    messages=query,
                    thread=thread,
                    settings=settings  # Pass execution settings
                ):
                    # Update thread from response
                    thread = response.thread

                    if isinstance(response, StreamingTextContent):
                        print(response.text, end="", flush=True)
                    elif hasattr(response, 'items'):
                        # Handle response items
                        for item in response.items:
                            if isinstance(item, StreamingTextContent):
                                print(item.text, end="", flush=True)

                print("\n")

        finally:
            # Cleanup
            print("\nCleaning up resources...")
            # Use the correct cleanup methods
            agents_client = azure_client.agents_client
            await agents_client.vector_stores.delete(vector_store.vector_store_id)
            await agents_client.files.delete(file_id)
            print("Cleaned up all resources.")

await main()

Uploaded file, file ID: assistant-Tyv4Raigc1oj9FNNC8oFUp
Created vector store, ID: vs_nasy306Yucezpzcx8EGMqCOi

Initialized Azure AI Agent for file search
Semantic Kernel agent created with STRICT file search enforcement.

User question: Can you explain Contoso's travel insurance coverage?

Agent response: 
Contoso’s travel insurance coverage includes:

• Medical emergencies  
• Trip cancellations  
• Lost baggage

User question: What is a Neural Network?

Agent response: 
I'm sorry, but the uploaded document does not contain the necessary information to answer that question.


Cleaning up resources...
Cleaned up all resources.


You should see output similar to:

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```